This notebook converts the clustering problem into a classification problem to serve the explainability purposes.

In [1]:
import sys
import pickle
import pandas as pd
sys.path.append('../')
from sklearn import svm
from pycaret.classification import *
from functions.explainability import *

In [2]:
# load data
data = pd.read_csv('../data/clustering_results/kmeans_2_results_hourly_categories_nooutliers3.csv')
data

,id,date,exertion_points,step_goal,minutes_below_zone_1,minutes_in_zone_1,steps,very_active_minutes,minutes_in_zone_2,minutes_in_zone_3,altitude,lightly_active_minutes,moderately_active_minutes,sedentary_minutes,exercises,exercise_duration,sleep_points,sleep_duration,calories,cluster
0,621e2e8e67b776a24055b564,2021-05-24 00:00:00,0.622928,0.0,0.936806,0.081134,0.017563,0.080685,0.000000,0.0,0.0,0.254701,0.083045,0.495139,0.074074,0.001865,0.810469,0.0,0.029382,1
1,621e2e8e67b776a24055b564,2021-05-24 01:00:00,0.622928,0.0,0.936806,0.081134,0.000000,0.080685,0.000000,0.0,0.0,0.254701,0.083045,0.495139,0.074074,0.001865,0.810469,0.0,0.002914,1
2,621e2e8e67b776a24055b564,2021-05-24 02:00:00,0.622928,0.0,0.936806,0.081134,0.046184,0.080685,0.000000,0.0,0.0,0.254701,0.083045,0.495139,0.074074,0.001865,0.810469,0.0,0.000729,1
3,621e2e8e67b776a24055b564,2021-05-24 03:00:00,0.622928,0.0,0.936806,0.081134,0.002661,0.080685,0.000000,0.0,0.0,0.254701,0.083045,0.495139,0.074074,0.001865,0.810469,0.0,0.012860,1
4,621e2e8e67b776a24055b564,2021-05-24 04:00:00,0.622928,0.0,0.936806,0.081134,0.000000,0.080685,0.000000,0.0,0.0,0.254701,0.083045,0.495139,0.074074,0.001865,0.810469,0.0,0.003315,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105551,621e375b67b776a240290cdc,2021-07-31 19:00:00,0.720932,0.0,0.568056,0.302053,0.008160,0.056235,0.003195,0.0,0.1,0.545299,0.221453,0.629167,0.185185,0.003484,0.680095,0.0,0.011658,1
105552,621e375b67b776a240290cdc,2021-07-31 20:00:00,0.720932,0.0,0.568056,0.302053,0.057123,0.056235,0.003195,0.0,0.1,0.545299,0.221453,0.629167,0.185185,0.003484,0.680095,0.0,0.039601,1
105553,621e375b67b776a240290cdc,2021-07-31 21:00:00,0.720932,0.0,0.568056,0.302053,0.122760,0.056235,0.003195,0.0,0.1,0.545299,0.221453,0.629167,0.185185,0.003484,0.680095,0.0,0.068017,1
105554,621e375b67b776a240290cdc,2021-07-31 22:00:00,0.720932,0.0,0.568056,0.302053,0.059429,0.056235,0.003195,0.0,0.1,0.545299,0.221453,0.629167,0.185185,0.003484,0.680095,0.0,0.052479,1


In [3]:
# data splitting 
train_data, test_data = train_test_split_per_user(data)
fold_groups = train_data.id

# training data shaping and cleaning
train_data.to_pickle('../data/explainability_input/train_data.pkl')
train_data = train_data.drop(columns=['id', 'date'])
x_train_data = train_data.drop(columns=['cluster'])
y_train_data = train_data['cluster']

# testing data shaping and cleaning
test_data.to_pickle('../data/explainability_input/test_data.pkl')
test_data = test_data.drop(columns=['id', 'date'])
x_test_data = test_data.drop(columns=['cluster'])
y_test_data = test_data['cluster']

In [4]:
# run pycaret to find the best claasiifer
s = setup(data=train_data, target='cluster', session_id=123, preprocess=False,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data)
best = compare_models()

,Description,Value
0,Session id,123
1,Target,cluster
2,Target type,Binary
3,Original data shape,"(105556, 18)"
4,Transformed data shape,"(105556, 18)"
5,Transformed train set shape,"(75248, 18)"
6,Transformed test set shape,"(30308, 18)"
7,Numeric features,17


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.9974,0.0000,0.9964,0.9984,0.9974,0.9947,0.9947,0.1100
lr,Logistic Regression,0.9970,1.0000,0.9962,0.9977,0.9970,0.9939,0.9939,1.3100
et,Extra Trees Classifier,0.9915,0.9998,0.9956,0.9872,0.9914,0.9828,0.9828,0.6300
ada,Ada Boost Classifier,0.9909,0.9993,0.9906,0.9911,0.9909,0.9817,0.9817,1.2067
gbc,Gradient Boosting Classifier,0.9889,0.9995,0.9897,0.9882,0.9890,0.9777,0.9777,3.3667
knn,K Neighbors Classifier,0.9839,0.9866,0.9884,0.9790,0.9836,0.9677,0.9678,7.5000
rf,Random Forest Classifier,0.9825,0.9995,0.9919,0.9738,0.9826,0.9649,0.9654,1.1400
dt,Decision Tree Classifier,0.9736,0.9743,0.9843,0.9631,0.9735,0.9470,0.9474,0.1433
lda,Linear Discriminant Analysis,0.9729,0.9980,0.9673,0.9778,0.9725,0.9454,0.9455,0.1533
ridge,Ridge Classifier,0.9727,0.0000,0.9668,0.9781,0.9724,0.9451,0.9453,0.0867


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [4]:
# use the SVC classifier 
svc = svm.SVC(probability=True, kernel='linear')  # default parameters
svc.fit(x_train_data, y_train_data)

# save the model
with open('../models/svc.pkl', 'wb') as file:
    pickle.dump(svc, file)